# Model exploration
This serves as notebook to look at different metabolites or reactions or ... in the GEMs present in this repository. 

In [11]:
# imports
import cobra
from libsbml import *
import refinegems as rg
import pandas as pd

RefineGEMs provides a couple of functions that make life a bit easier, so they will be used here. I still loaded cobra and libSBML to ensure that I have access to everything.

First we load our model with cobrapy.

In [2]:
model = rg.load_model_cobra('../models/Cstr_14.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


### Focus: Protocatechuic acid (PCA)
PCA is part of the CGXII medium which is of interest at the moment because this might be the minimal medium we need for more laboratory experiments.

In [3]:
model.metabolites.get_by_id('34dhbz_c')

Metabolite identifier,34dhbz_c
Name,"3,4-Dihydroxybenzoate"
Memory address,0x07f93d867cdc0
Formula,C7H5O4
Compartment,c
In 2 reaction(s),"PCADYOX, VNTDM"


Now we have a closer look at the two reactions that 34dhbz participates in.

In [4]:
model.reactions.get_by_id('PCADYOX')

Reaction identifier,PCADYOX
Name,
Memory address,0x07f93e99a4e80
Stoichiometry,"34dhbz_c + o2_c --> CCbuttc_c + 2.0 h_c 3,4-Dihydroxybenzoate + O2 O2 --> Cis,cis-Butadiene-1,2,4-tricarboxylate + 2.0 H+"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [5]:
model.reactions.get_by_id('VNTDM')

Reaction identifier,VNTDM
Name,
Memory address,0x07f93a91a0af0
Stoichiometry,"h_c + nadh_c + o2_c + vanlt_c --> 34dhbz_c + fald_c + h2o_c + nad_c H+ + Nicotinamide adenine dinucleotide - reduced + O2 O2 + Vanillate --> 3,4-Dihydroxybenzoate + Formaldehyde + H2O H2O + Nicotinamide adenine dinucleotide"
GPR,
Lower bound,0.0
Upper bound,1000.0


In strain 14 VNTDM produces 34dhbz, PCADYOX consumes 34dhbz. There exists no uptake reaction. In the other strains (15, 16, 17) 34dhbz is not even present.

In [81]:
len(cobra.medium.find_boundary_types(model, 'sink'))
cobra.medium.find_boundary_types(model, 'sink')
cobra.flux_analysis.loopless_solution(model)

minimal = cobra.medium.minimal_medium(model).to_dict()

In [82]:
cgxlab = rg.load.load_medium_from_db('/Users/baeuerle/Organisation/Masterarbeit/refinegems/data/media_db.csv', 'CGXlab')['BiGG_EX']
len(cgxlab)

20

In [83]:
with model:
    medium = model.medium
    new_medium = {i: 10.0 for i in cgxlab}
    new_medium.pop('EX_ni2_e')
    new_medium.pop('EX_na1_e')
    new_medium.pop('EX_34dhbz_e')
    try:
        if (new_medium['EX_o2_e'] == 10.0):
            new_medium['EX_o2_e'] = 20.0
    except KeyError:
        print('No Oxygen Exchange Reaction')
        pass
    model.medium = new_medium
    sol = model.optimize()
    print(model.medium)
    secretion = []
    uptake = []
    for index, value in sol.fluxes.items():
        if value > 0:
            secretion.append(index)
        if "EX" in index:
            if value < 0:
                uptake.append(index)
sol

{'EX_btn_e': 10.0, 'EX_ca2_e': 10.0, 'EX_cl_e': 10.0, 'EX_cu2_e': 10.0, 'EX_fe2_e': 10.0, 'EX_glc__D_e': 10.0, 'EX_h2o_e': 10.0, 'EX_h_e': 10.0, 'EX_k_e': 10.0, 'EX_mg2_e': 10.0, 'EX_mn2_e': 10.0, 'EX_nh4_e': 10.0, 'EX_o2_e': 20.0, 'EX_pi_e': 10.0, 'EX_so4_e': 10.0, 'EX_urea_e': 10.0, 'EX_zn2_e': 10.0}


,fluxes,reduced_costs
12DGR120tipp,0.0,0.000000e+00
12DGR140tipp,0.0,0.000000e+00
12DGR141tipp,0.0,0.000000e+00
12DGR160tipp,0.0,0.000000e+00
12DGR161tipp,0.0,0.000000e+00
...,...,...
EX_zn2_e,0.0,0.000000e+00
Growth,0.0,1.055350e-16
ATPM,0.0,5.009806e-18
APCPT,0.0,-1.261869e-30


Determine unused metabolites.

In [80]:
not_used = list(set(cgxlab)-set(uptake))

['EX_ca2_e',
 'EX_mn2_e',
 'EX_fe2_e',
 'EX_glc__D_e',
 'EX_cl_e',
 'EX_34dhbz_e',
 'EX_mg2_e',
 'EX_so4_e',
 'EX_zn2_e',
 'EX_pi_e',
 'EX_cu2_e',
 'EX_btn_e',
 'EX_ni2_e',
 'EX_h2o_e',
 'EX_urea_e',
 'EX_na1_e',
 'EX_k_e',
 'EX_nh4_e',
 'EX_h_e',
 'EX_o2_e']

In [79]:
new = list(set(cgxlab)-set(not_used))
with model:
    med = model.medium
    new_medium = {i: 10.0 for i in new}
    new_medium['EX_o2_e'] = 20.0
    model.medium = new_medium
    sol = model.optimize()
sol.objective_value

0.0